In [ ]:
def firn_equations(t, Vars):
    
    phi = Vars(1:N, :) 
    r2 = Vars(N+1:2*N, :) 
    A = Vars(2*N+1:3*N, :) 
    T = Vars(3*N+1:4*N, :)     
    H = Vars(4*N+1, :) 
        
### Compute gradients.
    dphidz = D1*phi 
    dr2dz = D1*r2 
    dAdz = D1*A 

### Compute stress.
    s_int = H*(1 - phi) 
    sigma = cumtrapz(z_h, s_int) 
    
### Compute the ice velocity.
    v_int = -(H/Ar).*sigma.**n.*phi.**m.*exp(lambda_c*T)./r2 
    w = cumtrapz(z_h, v_int) + 1*w_s 

### Column height.
    dHdt = w(end) - beta/(1 - phi(end))    
    
### Change in porosity.
    dphidt = (1/H)*(D1*((1 - phi).*w) + dHdt*z_h.*dphidz) 

### Change in square of the grain size.
    if sim_r   # (only if sim_r ==1) 
        dr2dt = (1/H)*(dHdt*z_h - w).*dr2dz + (1 - delta*r2).*exp(lambda_g*T) 
    else
        dr2dt = zeros(N,1) 
        

### Change in temperature.
    dTdt = zeros(N,1) 
    
### Change in age.
    dAdt = 1 + (1/H)*(dHdt*z_h - w).*dAdz 
#    dAdt = zeros(N,1) 
    
### Upper surface boundary conditions.
    dphidt(1) = 0 
    dr2dt(1) = 0 
    dAdt(1) = 0 

### Collected ODE vector.    
    dVarsdt = [dphidt  dr2dt  dAdt  dTdt  dHdt] 


In [45]:
class fs:   # short for firn simulation
    """The Doc String for this class"""
    


    
    def __init__(self,sim_label):
        import numpy as np
        self.sim_label = sim_label
        self.p = {}
        
    
    def setup(self,**kwargs):
        import numpy as np
        
        
        # 1. define default paramaters   
        self.p['b0_mpy'] = 0.1
        self.p['beta'] = 1
        self.p['nu'] = 1
        self.p['T_s_dim'] = 253.15
        self.p['plotting'] = 1
        self.p['saving_xt'] = 1
        self.p['plotting_period'] = 3000
        self.p['sampling_period'] = 20
        self.p['dz'] = 0.01
        self.p['t_total'] = 20
        self.p['save_dir'] = 'results_scratch'
        self.p['save'] = 1
        self.p['sim_T'] = True
        self.p['sim_r'] = True
        self.p['PauseGrainEvolution_t'] = np.nan
        self.p['z0'] = 100
        self.p['r_s_dim'] = 2.5000e-07   # grain size at the surface (0.5 mm)**2 
        self.p['phi_s'] = 0.5   
        self.p['n'] = 1
        self.p['simDuration'] = 10
        self.p['scaleDuration'] = False
        
        # 2. Replace any parameters that are defined by the user (in kwargs) 
        for key, values in kwargs.items():
            self.p[key] = values
            
            
        # 3. Define (or extract rom self.p) the dimensional parameters of the system.
        b0_mpy = self.p['b0_mpy']        # ice equivalent accumulation rate [m / yr]
        T_s_dim = self.p['T_s_dim']      # upper surface temperature [K]
        z_0 = self.p['z0']               # initial column height [m]
        dz_dim = self.p['dz']*z_0        # dimensional numerical grid spacing [m]
        r2_s_dim = self.p['r_s_dim']     # upper surface grain size [m**2] (0.5 mm)**2 
        r2_f = 0.01**2                        # maximum grain size [m**2] (1 cm)**2 
        phi_s = self.p['phi_s']          # upper surface porosity
        c_i = 2009                            # heat capacity [J / (kg K)]
        E_c = 60e3                            # compaction activation energy [J]
        E_g = 42e3                            # grain growth activation energy [J]
        k_c = 9.2e-9                          # flow parameter [kg?1 m3 s] from Arthern kc
        k_g = 1.3e-7/r2_f                     # grain growth rate constant [m**2 / s]
        m = 1                                 # porosity exponent
        n = self.p['n']                  # stress exponent
        kappa_0 = 2.1                         # thermal conductivity [W / (m K)], # arthern and wingham 1998 pg. 18
        G = 0.05                              # geothermal heat flux  [W/m**2]
       
        ## 4. Constants, scales and parameters
        ### 4.1 constants
        g = 9.81                           # acceleration due to gravity [m s**-2]
        spy = 24*365*3600                  # seconds per year
        R = 8.31                           # ideal gas constant
        rho_i = 918                        # ice density [kg / m**3]

        ### 4.2 Scaling parameters.
        b_0 = b0_mpy/spy 
        h_0 = z_0 
        r2_0 = (h_0*k_g*r2_f/b_0)*np.exp(-E_g/(R*T_s_dim)) 
        t_0 = h_0/b_0 
        T_0 = G*z_0/kappa_0 
        sigma_0 = g*rho_i*h_0 
        w_0 = b_0                 # scale of the vertical velocity is the accumulation rate


        ### 4.3 Non-dimensional parameters.
        lambda_c = E_c*T_0/(R*T_s_dim**2.0) 
        lambda_g = E_g*T_0/(R*T_s_dim**2.0) 
        gamma = (sigma_0/4.0)**(1-n)    # factor to account for non-linear rheology, under the default value of n = 1, gamma = 1 and it has no effect on the value of Ar
        Ar = r2_0/(k_c*t_0*sigma_0**np.exp(-E_c/(R*T_s_dim)))/gamma  # Arthern number
        Fl = h_0*G/(kappa_0*T_0) 
        Pe = rho_i*c_i*b_0*h_0/kappa_0 
        beta = self.p['beta'] 
        delta = r2_0/r2_f 
        nu = self.p['n']       # accumulation multiplier

        ## 5. Compute vertical velocity upper biundary conditon
        w_s = nu*beta/(1 - phi_s)  

        ## 6. Set up real space grid in height coordinates.
        z_init = np.arange(z_0,-dz_dim,-dz_dim)
        N = z_init.size
        
        ### Normalized depth coordinates.
        z_h = np.flip(z_init)/z_0 

        ## 7. Initial conditions
        ### Initial porosity
        phi_init = (1-z_h)*phi_s 

        ### Dimensional  grain size squared.
        if self.p['sim_r']:
            r2_hat_init = r2_s_dim/r2_0 + z_h 
        else:
            r2_hat_init = r2_s_dim/r2_0 + 0*z_h 
        

        ### Dimensionless temperature (mostly not used)
        T_hat_init = np.zeros(N) 

        ### Dimensionless firn age.
        A_hat_init = z_h 

        ### Initial conditions vector.
        #Vars_init = [phi_init  r2_hat_init  A_hat_init  T_hat_init  z_0/h_0] 

        ## 8. Define gradient operator
        ### Finite difference gradient operator using three point upwind scheme.
        #D1 = two_point_upwind_uni_D1(z_h(1), z_h(end), N, 1) 

        ## 9. Save outputs for use at the solution stage

        # 10. Add the newly created values to p
        self.p['GridNumber'] = N 
        #self.p['Gradient'] = D1
        self.p['spy'] = spy 
        self.p['lambda_c'] = lambda_c 
        self.p['lambda_g'] = lambda_g 
        self.p['PecletNumber'] = Pe 
        self.p['FluxNumber'] = Fl 
        self.p['ArthenNumber'] = Ar 
        #self.p['InitialConditions'] = Vars_init 
         
    def run(self):
        from scipy.integrate import solve_bvp
        
        if p.scaleDuration
            time = [0 p.simDuration/beta];
        else 
            time = [0 p.simDuration];


### Solve the system

        #options = odeset('Events', @SteadyState, 'RelTol',1e-8,'AbsTol', 1e-8);
        #[Time, Vars] = ode15s(@GovEq, time, Vars_init, options);
                sol = solve_bvp(firn_equations, bc_N,x,y,tol=0.00000001,bc_tol=0.00000001,verbose=2,max_nodes=10000)

        ### Collect the simulation variables.
        Phi = Vars(:, 1:N)';
        Rho = p.rho_i*(1 - Phi);
        GrainSize = Vars(:, N+1:2*N)';
        Age = Vars(:, 2*N+1:3*N)';
        Temp = Vars(:, 3*N+1:4*N)';
        Height = Vars(:,4*N+1);

        ### Un-normalized depth coordinates
        Depth = p.h_0*repmat(Height', N, 1)*repmat(z_h, 1, numel(Time));

        ### Compute the firn thickness as a function of time
        zeta830final = interp1(Phi(:,end),Depth(:,end)/p.h_0,1-830/p.rho_i);  # normalized


        ### Compute normalized ice velocity
        W = nan(N, numel(Time));
        Mass = nan(numel(Time),1);
        for i = 1:numel(Time)
            S_int = Height(i,1).*(1 - Phi(:,i));
            Sigma = cumtrapz(z_h, S_int);
            V_int = -(Height(i,1)/Ar).*Sigma.**n.*Phi(:,i).**m...
                .*exp(lambda_c*Temp(:,i))./GrainSize(:,i);
            W(:,i) = cumtrapz(z_h, V_int) + w_s;

        ### Compute mass in the column.
            Mass(i) = trapz(Depth(:,i), 1 - Phi(:,i));
        end

        ### Create output stucture.
        Out = struct('Time', Time, 'Rho', Rho, 'Temp', Temp, 'GrainSize', GrainSize,...
            'Sigma', Sigma, 'Phi', Phi, 'Age', Age, 'Depth', Depth, 'W', W, 'p', p,...
            'H', Height, 'Mass', Mass, 'zeta830final',zeta830final);
        
        
        

    
s1 = fs('figure_1_run')
s1.setup()

    

[100.  99.  98.  97.  96.  95.  94.  93.  92.  91.  90.  89.  88.  87.
  86.  85.  84.  83.  82.  81.  80.  79.  78.  77.  76.  75.  74.  73.
  72.  71.  70.  69.  68.  67.  66.  65.  64.  63.  62.  61.  60.  59.
  58.  57.  56.  55.  54.  53.  52.  51.  50.  49.  48.  47.  46.  45.
  44.  43.  42.  41.  40.  39.  38.  37.  36.  35.  34.  33.  32.  31.
  30.  29.  28.  27.  26.  25.  24.  23.  22.  21.  20.  19.  18.  17.
  16.  15.  14.  13.  12.  11.  10.   9.   8.   7.   6.   5.   4.   3.
   2.   1.   0.]


In [44]:
len(s1.params['z_h'])

1001

In [208]:
s1.setup(m='new_m',n='new_n',r_s_sim = 'new_value')


TypeError: unsupported operand type(s) for ^: 'float' and 'int'

In [183]:
s1.params


{'b0_mpy': 0.1,
 'beta': 1,
 'nu': 1,
 'T_s_dim': 253.15,
 'plotting': 1,
 'saving_xt': 1,
 'plotting_period': 3000,
 'sampling_period': 20,
 'dz': 0.01,
 't_total': 20,
 'save_dir': 'results_scratch',
 'save': 1,
 'sim_T': True,
 'sim_r': True,
 'PauseGrainEvolution_t': nan,
 'z0': 100,
 'r_s_dim': 2.5e-07,
 'phi_s': 0.5,
 'n': 'new_n',
 'simDuration': 10,
 'scaleDuration': 0,
 'm': 'new_m',
 'r_s_sim': 'new_value'}

In [184]:
s1.g

9.81

In [74]:
results

array([10.81, 11.81, 12.81])

In [116]:
dir({})

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [18]:
fs.phi_s = 0.4


In [20]:
fs.phi_s

0.4

In [23]:
fs.setupT()

TypeError: setupT() missing 1 required positional argument: 'self'